In [2]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
import pandas as pd
from utils import show_recipe
is_user_input = False

C:\Users\louis\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
model_path = "model/finetuned_recipenlg_3"
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForCausalLM.from_pretrained(model_path)

device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)

if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

if not is_user_input:
    prompt = (
        "Give me a healthy recipe using the following ingredients (you don't have to use all of them):"
        "<RECIPE_START>"
        "<INPUT_START>chocolate<NEXT_INPUT>pickles<NEXT_INPUT>beef<NEXT_INPUT>pear<INPUT_END>"
    )

inputs = tokenizer(
    prompt,
    return_tensors="pt",
    truncation=True,
    max_length=512,
    return_attention_mask=True,
)
input_ids = inputs["input_ids"].to(device)
attention_mask = inputs["attention_mask"].to(device)

outputs = model.generate(
    input_ids=input_ids,
    attention_mask=attention_mask,
    max_length=512,
    num_beams=2,
    no_repeat_ngram_size=2,
    early_stopping=True,
    eos_token_id=tokenizer.convert_tokens_to_ids("<RECIPE_END>"),
)

raw_recipe = tokenizer.decode(outputs[0])
show_recipe(raw_recipe)

Setting `pad_token_id` to `eos_token_id`:50266 for open-end generation.


## Recipe Title: Chocolate Salad

Inputs (base ingredients):
   - chocolate
   - pickles
   - beef
   - pear

Ingredients:
   - 1 lb. chocolate
   - 1/2 c. pickles, chopped
   - 2 cans beef or pork sausage (hot or mild)
   - 3 to 4 slices pear

Instructions:
   1. Melt chocolate in double boiler.
   2. Add pickle and sausage. Mix well and add pear slices.
